In [3]:
#%env GOOGLE_APPLICATION_CREDENTIALS=/home/pitfox/data/spark-container-dev-f5d53ab2439c.json

In [4]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/home/pitfox/data/spark-container-dev-f5d53ab2439c.json


In [5]:
!echo $SPARK_HOME

/opt/spark


In [6]:
!cat $SPARK_HOME/conf/spark-defaults.conf

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Default system properties included when running spark-submit.
# This is useful for setting default environmental settings.

# Example:
# spark.master                     spark://master:7077
# spark.eventLog.enable

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkFiles
import pyspark.sql.functions as f
import re

In [2]:
#spark.stop()

In [3]:
spark = SparkSession \
    .builder \
    .appName("My App") \
    .config("spark.jars", "/opt/spark/jars/spark-bigquery-with-dependencies_2.12-0.20.0.jar") \
    .getOrCreate()

In [4]:
# spark.sparkContext._jvm.scala.util.Properties.versionString()

In [27]:
category = ['AMAZON_FASHION','All_Beauty']

In [28]:
url = f"http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/{category[1]}_5.json.gz"

In [29]:
spark.sparkContext.addFile(url)

In [30]:
df = spark.read.json("file://"+SparkFiles.get(f"{category[1]}_5.json.gz"))

In [31]:
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- style: struct (nullable = true)
 |    |-- Color:: string (nullable = true)
 |    |-- Design:: string (nullable = true)
 |    |-- Flavor:: string (nullable = true)
 |    |-- Scent Name:: string (nullable = true)
 |    |-- Size:: string (nullable = true)
 |    |-- Style Name:: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- vote: string (nullable = true)



In [32]:
df.show(10)

+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|      asin|image|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|               style|             summary|unixReviewTime|verified|vote|
+----------+-----+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------------+--------------+--------+----+
|B0000530HU| null|    5.0|As advertised. Re...| 09 1, 2016|A3CIUOJXQ5VDQ2|            Shelly F|{null, null,  Cla...|          Five Stars|    1472688000|    true|null|
|B0000530HU| null|    5.0|Like the oder and...|11 14, 2013|A3H7T87S984REU|        houserules18|{null, null,  Cla...|   Good for the face|    1384387200|    true|null|
|B0000530HU| null|    1.0|I bought this to ...|08 18, 2013|A3J034YH7UG4KT|                Adam|{null, null,  Cla...|        Smells awful|    1376784000|    true|null

In [33]:
df = df.dropDuplicates() 

In [34]:
df = df.withColumn('review_wordCount', f.size(f.split(f.col('reviewText'), ' ')))

In [35]:
df.registerTempTable("dataframe")

In [39]:
sql_script = f"""select 
          '{category[1]}' as category,
          asin || '-' || reviewerID || row_number() OVER (PARTITION BY asin, reviewerID ORDER BY unixReviewTime asc) as review_ID,
          asin as product_ID,
          reviewerID as reviewer_ID,
          overall as rating_out_of_5,
          summary as review_summary,
          reviewText as review_text,
          review_wordCount as review_word_count,     
          '{url}' as source_url
        from dataframe"""

In [40]:
output = spark.sql(sql_script)

In [41]:
output.show(3)

+----------+--------------------+----------+--------------+---------------+--------------+--------------------+-----------------+--------------------+
|  category|           review_ID|product_ID|   reviewer_ID|rating_out_of_5|review_summary|         review_text|review_word_count|          source_url|
+----------+--------------------+----------+--------------+---------------+--------------+--------------------+-----------------+--------------------+
|All_Beauty|B00006L9LC-A1WMNC...|B00006L9LC|A1WMNCCFKM2A4Z|            5.0|    Five Stars|Great product gre...|                4|http://deepyeti.u...|
|All_Beauty|B00006L9LC-A1Z14Z...|B00006L9LC|A1Z14Z7HOM429U|            5.0|    Five Stars|Yeah, the huge si...|               13|http://deepyeti.u...|
|All_Beauty|B00006L9LC-A3CFIN...|B00006L9LC|A3CFINYERTHF9Q|            5.0|    Five Stars|       Great shampoo|                2|http://deepyeti.u...|
+----------+--------------------+----------+--------------+---------------+--------------+----

In [42]:
# Update to your GCS bucket
gcs_bucket = f'amazon_reviews_bucket'

gcs_filepath = f'gs://amazon_reviews_bucket'

In [43]:
output.write \
  .format("bigquery") \
  .option("temporaryGcsBucket",gcs_bucket) \
  .mode("append") \
  .save("amazon_reviews.categoryFilesSmall")

In [44]:
# output.write.partitionBy("product_ID").csv(gcs_filepath)

In [45]:
# output.write \
#   .mode('overwrite') \
#   .csv(gcs_filepath)